# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [1]:
import os
import sys

# In this case the local root of the repo is our working directory
DIRECTORY = './'
font = 'arial'

import tensorflow as tf
from Library.Build_Model import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(filename, Stats, model, time): 
    # printing Stats
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))

# Hybrid models

# Examples of trainable AMN models (neural and mechanistic)

# Using FBA simulated training sets or experimental datasets

Each cell is showing a different setup: 

- using either E. coli core simulations, iML1515 simulations, or the experimental dataset as the training set;
- using either LP, QP or Wt as the mechanistic layer of the AMNs

### AMN with QP solver on iML1515 FBA simulated training set

In [3]:
# Create, train and evaluate AMN_QP o models with FBA simulated training set for E. coli core
# with EB or UB with a mechanistic layer

# What you can change 
seed = 10
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'mediabotJLF1_UB' # can change EB by UB
timestep = 4
# End of What you can change
  
# Create model 90% for training 10% for testing
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
                     objective=['BIOMASS_Ec_iML1515_core_75p37M'],  
                     model_type='AMN_QP', 
                     timestep = timestep, learn_rate=0.01,
                     scaler=True,
                     n_hidden = 1, hidden_dim = 50,
                     epochs=50, xfold=5,
                     verbose=True)
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
reservoir.save(reservoirfile)
reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir, verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)
    

number of reactions:  507 507
number of metabolites:  1078
filtered measurements size:  1
training file: ./Dataset_model/mediabotJLF1_UB
model type: AMN_QP
model scaler: 1.0
model input dim: 34
model output dim: 1
model medium bound: UB
timestep: 4
training set size (66, 34) (66, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
gradient learn rate: 0.01
gradient decay rate: 0.9
training epochs: 50
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
nbr parameters: 27607
---------- 1
---------- 1
train = 0.98 test = 0.95 loss-train = 0.000489 loss-test = 0.000542 iter=0
nbr parameters: 27607
---------- 1
---------- 1
train = 0.98 test = 0.98 loss-train = 0.000585 loss-test = 0.000488 iter=0
nbr parameters: 27607
---------- 1
---------- 1
train = 0.98 test = 0.98 loss-train = 0.000545 loss-test = 0.000702 iter=0
nbr parameters: 27607
----------

### AMN with LP solver on iML1515 FBA simulated training set

In [3]:
# Create, train and evaluate AMN_LP models with FBA simulated training set for E. coli core
# with UB with a mechanistic layer

# What you can change 
seed = 10
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'mediabotJLF1_UB' # can change EB by UB
timestep = 4
# End of What you can change
  
# Create model 90% for training 10% for testing
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
                     objective=['BIOMASS_Ec_iML1515_core_75p37M'],  
                     model_type='AMN_LP', 
                     timestep = timestep, learn_rate=1.0e-6,
                     scaler=True,
                     n_hidden = 1, hidden_dim = 50,
                     epochs= 50, xfold=5,
                     verbose=True)
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), 
                      replace=False)
# LP keeps track of boundary fluxes in b_int and b_ext
# and these are different in EB or UB modes
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
btest = model.b_ext[ID,:] if model.mediumbound == 'UB' else model.b_int[ID,:]
bint  = model.b_int if model.mediumbound == 'UB' else model.b_ext
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
btrain = np.delete(model.b_ext, ID, axis=0) if model.mediumbound == 'UB' \
else np.delete(model.b_int, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y, model.b_ext, model.b_int = Xtrain, Ytrain, btrain, bint
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
reservoir.save(reservoirfile)
# reservoir.load(reservoirfile) 
# # Issue for loading AMN-LP models: takes about 15 minutes (when QP takes a few)
# reservoir.printout()
# if len(Xtest) > 0:
#     start_time = time.time()
#     reservoir.X, reservoir.Y = Xtest, Ytest
#     reservoir.b_ext, reservoir.b_int =  btest, bint
#     X, Y = model_input(reservoir,verbose=False)
#     pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
#     delta_time = time.time() - start_time
#     printout('Test set', stats, model, delta_time)
    

number of reactions:  507 507
number of metabolites:  1078
filtered measurements size:  1
training file: ./Dataset_model/mediabotJLF1_UB
model type: AMN_LP
model scaler: 1.0
model input dim: 34
model output dim: 1
model medium bound: UB
timestep: 4
training set size (66, 34) (66, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
training epochs: 50
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
nbr parameters: 165170
---------- 1
ERROR:
R2 calculation failed: Input contains NaN, infinity or a value too large for dtype('float32').
looping bad training iter=0 r2=-1.0000
---------- 1
ERROR:
R2 calculation failed: Input contains NaN, infinity or a value too large for dtype('float32').
train = -1.00 test = -1.00 loss-train =    nan loss-test = nan iter=0
nbr parameters: 165170
---------- 1
ERROR:
R2 calculation failed: Input contains NaN, infi

### AMN with QP solver on iML1515 FBA simulated training set

In [5]:
# Create, train and evaluate AMN_QP models with FBA simulated training set for iML1515
# with EB or UB with a mechanistic layer 
# This cell takes several hours to execute

# What you can change 
seed = 1
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'mediabotJLF1_UB' # can change EB by UB
timestep = 4
# End of What you can change
  
# Create model 
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_QP',
              scaler = True,
              timestep = timestep, learn_rate=0.01,
              n_hidden = 1, hidden_dim = 500,
              epochs = 25, xfold = 5, 
              verbose=True) 
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
reservoir.save(reservoirfile)
reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir, verbose=False)
    reservoir.model.b_ext = btest
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)
 

number of reactions:  507 507
number of metabolites:  1078
filtered measurements size:  1
training file: ./Dataset_model/mediabotJLF1_UB
model type: AMN_QP
model scaler: 1.0
model input dim: 34
model output dim: 1
model medium bound: UB
timestep: 4
training set size (66, 34) (66, 1)
nbr hidden layer: 1
hidden layer size: 500
activation function: relu
gradient learn rate: 0.01
gradient decay rate: 0.9
training epochs: 25
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
AMN scaler 39.5959595959596
QP input shape (60, 34) (60, 4)
-------train [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 17 18 19 22 23 24 25 26 27
 28 29 30 32 34 35 36 37 38 39 40 42 44 47 49 51 52 53 54 55 56 57 58 59] (48, 34) (48, 4)
-------test  [ 4 16 20 21 31 33 41 43 45 46 48 50] (12, 34) (12, 4)
----------------------------------- AMN_QP
Dense layer n_hidden, hidden_dim, output_dim, activa

### AMN with LP solver on iML1515 FBA simulated training set

In [6]:
# Create, train and evaluate AMN_LP models with FBA simulated training set for iML1515
# with UB with a mechanistic layer 
# This cell takes several hours to execute

# What you can change 
seed = 1
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'iML1515_UB'
timestep = 4
# End of What you can change
  
# Create model 
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_LP',
              scaler = True,
              timestep = timestep, learn_rate=1.0e-6,
              n_hidden = 1, hidden_dim = 250,
              epochs = 25, xfold = 5, 
              verbose=True) 

ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), 
                      replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
btest = model.b_ext[ID,:]
bint = model.b_int
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
btrain = np.delete(model.b_ext, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y, model.b_ext, model.b_int = Xtrain, Ytrain, btrain, bint
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
reservoir.save(reservoirfile)
# reservoir.load(reservoirfile)
# Issue for loading AMN-LP models: takes about 15 minutes (when QP takes a few)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    reservoir.b_ext, reservoir.b_int =  btest, bint
    X, Y = model_input(reservoir,verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)


number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
training file: ./Dataset_model/iML1515_UB
model type: AMN_LP
model scaler: 1.0
model input dim: 38
model output dim: 1
model medium bound: UB
timestep: 4
training set size (11000, 38) (11000, 1)
nbr hidden layer: 1
hidden layer size: 250
activation function: relu
training epochs: 25
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
AMN scaler 11.0
LP input shape (9900, 3235) (9900, 4)
-------train [   0    1    3 ... 9897 9898 9899] (7920, 3235) (7920, 4)
-------test  [   2    4    6 ... 9883 9891 9893] (1980, 3235) (1980, 4)
----------------------------------- AMN_LP
Dense layer n_hidden, hidden_dim, output_dim, activation, trainable: 1 250 550 relu True
Dense layer n_hidden, hidden_dim, output_dim, activation, trainable: 1 250 2716 linear True
AMN output shapes for PoutV,

### AMN-Wt on iML1515 FBA simulated training set

In [3]:
# Create, train and evaluate AMN_Wt models with FBA simulated training set for E. coli core
# with UB (not working with M1 chips)

# What you can change 
seed = 10
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'mediabotJLF1_UB' # can change EB by UB
timestep = 4
# End of What you can change
  
# Create model 90% for training 10% for testing
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
                     objective=['BIOMASS_Ec_iML1515_core_75p37M'],  
                     model_type='AMN_Wt', 
                     timestep = timestep,
                     n_hidden = 1, hidden_dim = 50,
                     batch_size=6,
                     scaler=True,
                     train_rate=1e-2,
                     epochs=50, xfold=5,
                     verbose=True)
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
#reservoir.save(reservoirfile)
#reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir, verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)

number of reactions:  507 507
number of metabolites:  1078
filtered measurements size:  1
training file: ./Dataset_model/mediabotJLF1_UB
model type: AMN_Wt
model scaler: 1.0
model input dim: 34
model output dim: 1
model medium bound: UB
timestep: 4
training set size (66, 34) (66, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
training epochs: 50
training regression: True
training learn rate: 0.01
training dropout: 0.25
training batch size: 6
training validation iter: 0
training xfold: 5
training early stopping: False
nbr parameters: 550487
---------- 1
looping bad training iter=0 r2=0.0083
---------- 1
train = 0.01 test = -0.02 loss-train = 0.001573 loss-test = 0.001572 iter=0
nbr parameters: 550487
---------- 1
looping bad training iter=0 r2=0.0009
---------- 1
train = 0.00 test = -0.02 loss-train = 0.003072 loss-test = 0.003073 iter=0
nbr parameters: 550487
---------- 1
---------- 1
train = 0.85 test = 0.87 loss-train = 0.001951 loss-test = 0.002331 iter=0
nbr

In [6]:
X.shape, Y.shape, Xtest.shape, Ytest.shape, model.batch_size

((10, 4, 34), (6, 4), (6, 34), (6, 1), 5)

### AMN-QP with experimental training set

In [ ]:
# Create, train and evaluate AMN_QP models on experimental training set with UB 
# Repeat the process with different seeds
# This cell takes several hours to execute

Maxloop, Q2, PRED = 3, [], []

for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop+1
    np.random.seed(seed=seed)  
    trainname = 'mediabotJLF1_UB' 
    timestep = 2
    # End of What you can change
  
    # Create model 100% for training 0% for testing
    trainingfile = DIRECTORY+'Dataset_model/'+trainname
    model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_QP',
              scaler = True,
              timestep = timestep, learn_rate=0.001,
              n_hidden = 1, hidden_dim = 500,
              #train_rate = 1.0e-2,
              epochs = 10, xfold = 10, 
              verbose=True) 

    # Train and evaluate
    reservoirname = trainname +'_'+model.model_type
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred[:,0])

# Save in Result folder
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_PRED.csv'
np.savetxt(filename, PRED, delimiter=',')


### AMN-LP with experimental training set

In [ ]:
# Create, train and evaluate AMN_QP models on experimental training set with UB 
# Repeat the process with different seeds
# This cell takes several hours to execute

Maxloop, Q2, PRED = 3, [], []

for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop+1
    np.random.seed(seed=seed)  
    trainname = 'mediabotJLF1_UB' 
    timestep = 4
    # End of What you can change
  
    # Create model 100% for training 0% for testing
    trainingfile = DIRECTORY+'Dataset_model/'+trainname
    model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_LP',
              scaler = True,
              timestep = timestep, learn_rate=0.001,
              n_hidden = 1, hidden_dim = 500,
              #train_rate = 1.0e-2,
              epochs = 1000, xfold = 10, 
              verbose=True) 

    # Train and evaluate
    reservoirname = trainname +'_'+model.model_type
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred[:,0])

# Save in Result folder
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_PRED.csv'
np.savetxt(filename, PRED, delimiter=',')


### AMN-Wt with experimental training set

In [ ]:
# Create, train and evaluate AMN_Wt models on experimental training set with UB 
# Repeat the process with different seeds
# This cell takes several hours to execute

Maxloop, Q2, PRED = 3, [], []

for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop+1
    np.random.seed(seed=seed)  
    trainname = 'mediabotJLF1_UB' 
    timestep = 4
    # End of What you can change
  
    # Create model 100% for training 0% for testing
    trainingfile = DIRECTORY+'Dataset_model/'+trainname
    model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_Wt',
              scaler = True,
              timestep = timestep,
              n_hidden = 1, hidden_dim = 500,
              #train_rate = 1.0e-2,
              epochs = 1000, xfold = 10, 
              verbose=True) 

    # Train and evaluate
    reservoirname = trainname +'_'+model.model_type
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred[:,0])

# Save in Result folder
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_PRED.csv'
np.savetxt(filename, PRED, delimiter=',')
